In [ ]:
#import libraries
import numpy as np
import pandas
import sklearn.preprocessing
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report
import time
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D

In [ ]:
#import data
data_small = pandas.read_csv('data28.csv')
y = data_small['label']
X = data_small[data_small.columns[1:]]

# 70-30 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
#preprocess

indices_to_keep=~X_train.isin([np.nan,np.inf,-np.inf]).any(1)

X_train=X_train[indices_to_keep]
y_train=y_train[indices_to_keep]

indices_to_keep_test=~X_test.isin([np.nan,np.inf,-np.inf]).any(1)

X_test=X_test[indices_to_keep_test]
y_test=y_test[indices_to_keep_test]

In [ ]:
length = len(X_train)
X_train=X_train.iloc[:,:].values.reshape(length,28,28,1)
length_t = len(X_test)
X_test = X_test.iloc[:,:].values.reshape(length_t,28,28,1)

In [ ]:
y_train_n = []
for item in y_train:
    if item == "Whole":
        y_train_n.append(0)
    elif item == "Half":
        y_train_n.append(1)
    elif item == "Quarter":
        y_train_n.append(2)
    elif item == "Eight":
        y_train_n.append(3)
    elif item == "Sixteenth":
        y_train_n.append(4)
y_train_n = keras.utils.to_categorical(y_train_n, num_classes=5)

In [ ]:
y_test_n = []
for ele in y_test:
    if ele == "Whole":
        y_test_n.append(0)
    elif ele == "Half":
        y_test_n.append(1)
    elif ele == "Quarter":
        y_test_n.append(2)
    elif ele == "Eight":
        y_test_n.append(3)
    elif ele == "Sixteenth":
        y_test_n.append(4)
y_test_n = keras.utils.to_categorical(y_test_n, num_classes=5)

In [ ]:
#CNN
model = Sequential()
model.add(Convolution2D(32,3,data_format="channels_last",activation="relu",input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(50,11,activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adadelta', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 3, 3, 50)          193650    
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 1, 1, 50)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_17 (Dense)             (None, 5)               

In [ ]:
model.fit(X_train, y_train_n, validation_data = (X_test, y_test_n), epochs=50)

Epoch 1/50
125/125 [==============================] - 5s 43ms/step - loss: 0.1081 - accuracy: 0.9705 - val_loss: 0.2376 - val_accuracy: 0.9250
Epoch 2/50
125/125 [==============================] - 5s 43ms/step - loss: 0.1076 - accuracy: 0.9745 - val_loss: 0.2346 - val_accuracy: 0.9290
Epoch 3/50
125/125 [==============================] - 5s 44ms/step - loss: 0.1071 - accuracy: 0.9730 - val_loss: 0.2405 - val_accuracy: 0.9230
Epoch 4/50
125/125 [==============================] - 5s 43ms/step - loss: 0.1066 - accuracy: 0.9750 - val_loss: 0.2355 - val_accuracy: 0.9270
Epoch 5/50
125/125 [==============================] - 5s 43ms/step - loss: 0.1051 - accuracy: 0.9750 - val_loss: 0.2340 - val_accuracy: 0.9280
Epoch 6/50
125/125 [==============================] - 5s 43ms/step - loss: 0.1047 - accuracy: 0.9740 - val_loss: 0.2299 - val_accuracy: 0.9260
Epoch 7/50
125/125 [==============================] - 6s 44ms/step - loss: 0.1041 - accuracy: 0.9755 - val_loss: 0.2315 - val_accuracy: 0.9250

In [ ]:
model.predict(X_test)

array([[2.13201929e-05, 5.81134930e-02, 9.35856402e-01, 5.18457592e-03,
        8.24177056e-04],
       [2.04255957e-05, 1.96048475e-07, 4.83509572e-03, 2.34344825e-01,
        7.60799408e-01],
       [7.67301462e-05, 2.05981178e-05, 1.19669705e-04, 7.66640306e-01,
        2.33142763e-01],
       ...,
       [3.23594418e-06, 5.94411176e-05, 1.21282364e-05, 6.40061125e-02,
        9.35919046e-01],
       [1.34581060e-04, 9.85395849e-01, 1.28430277e-02, 1.57879980e-03,
        4.77127396e-05],
       [3.14491044e-05, 5.04059310e-04, 9.98670220e-01, 4.06294275e-04,
        3.87986831e-04]], dtype=float32)

In [ ]:
model.evaluate(X_test, y_test_n)

32/32 [==============================] - 0s 9ms/step - loss: 0.2061 - accuracy: 0.9420


[0.20614472031593323, 0.9419999718666077]